1,处理全国指数趋势表

In [1]:
import numpy as np
import pandas as pd

D:\SoftwareInstallation\python_Anaconda\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# 读取百度指数表
index = pd.read_excel('././数据/baidu_index_0625.xlsx')
prov_id = pd.read_excel('././././数据/province_id.xlsx')
city_id = pd.read_excel('././././数据/province_id.xlsx')



In [7]:
# 处理缺失值
# print(index)
index = index.fillna(0)
print(index)


        id  keyword  _index       date
0        1       缤智  2699.0 2018-12-01
1        2       缤智  2767.0 2018-12-02
2        3       缤智  2866.0 2018-12-03
3        4       缤智  2872.0 2018-12-04
4        5       缤智  2739.0 2018-12-05
5        6       缤智  2706.0 2018-12-06
6        7       缤智  2584.0 2018-12-07
7        8       缤智  2573.0 2018-12-08
8        9       缤智  2502.0 2018-12-09
9       10       缤智  2954.0 2018-12-10
10      11       缤智  2635.0 2018-12-11
11      12       缤智  2430.0 2018-12-12
12      13       缤智  2606.0 2018-12-13
13      14       缤智  2734.0 2018-12-14
14      15       缤智  2642.0 2018-12-15
15      16       缤智  2612.0 2018-12-16
16      17       缤智  2915.0 2018-12-17
17      18       缤智  2882.0 2018-12-18
18      19       缤智  2685.0 2018-12-19
19      20       缤智  2489.0 2018-12-20
20      21       缤智  2691.0 2018-12-21
21      22       缤智  2653.0 2018-12-22
22      23       缤智  2643.0 2018-12-23
23      24       缤智  3100.0 2018-12-24
24      25       缤智  2811

In [8]:
# 对data字段进行格式化处理
index['date']=pd.to_datetime(index['date'])
index['date']=index['date'].dt.strftime('%B')


In [9]:
# 对关键词字段进行统一处理
index['keyword'] = index['keyword'].apply(lambda x:x.strip(' \r\n\t').upper())

In [10]:
# 根据keyword，date对搜索指数进行分类汇总求和
new_index_mean = index.groupby(['keyword','date'])['_index'].sum()


In [11]:
# 展示结果
new_index_mean

keyword     date    
IX25        April        29144.0
            December     32422.0
            February     28511.0
            January      32204.0
            June           882.0
            March        30081.0
            May          27164.0
            November     30810.0
            October      31611.0
T-CROSS     April       150414.0
            December         0.0
            February         0.0
            January          0.0
            June         29702.0
            March        77619.0
            May         120753.0
            November         0.0
            October          0.0
XR-V        April         6656.0
            December      7034.0
            February      6835.0
            January       6864.0
            June           207.0
            March         7227.0
            May           7046.0
            November      6745.0
            October       7421.0
三菱ASX       April         7067.0
            December      6015.0
            February  

2、读取省份搜索指数数据

In [14]:
# 读取省份搜索指数数据
prov_index = pd.read_excel('././././数据/province_index_0625.xlsx')


In [15]:
# 同样对日期进行格式化处理
prov_index['date'] = prov_index['date'].apply(lambda x:x.split("|")[0])
prov_index['date'] = pd.to_datetime(prov_index['date'])# 格式化日期
prov_index['date'] = prov_index['date'].dt.strftime('%B') # 将日期转换成月份


In [16]:
# 对字段进行统一大写和去空格处理
prov_index['keyword'] = prov_index['keyword'].apply(lambda x:x.strip(" \r\n\t").upper())
prov_index.head()

,id,keyword,prov,prov_index,date
0,1,缤智,913,1000,December
1,2,缤智,901,272,December
2,3,缤智,917,246,December
3,4,缤智,916,234,December
4,5,缤智,920,203,December


In [17]:
# 根据keyword，date对搜索指数进行分类汇总，得出假的全国搜索指数
prov_index_sum = prov_index.groupby(['keyword','date'])['prov_index'].sum()


In [19]:
# 分类汇总后得到的是一个Series，因为需要合并DataFrame，所以进行reset_index()处理
prov_index_sum = prov_index_sum.reset_index()
prov_index_sum.head()

,keyword,date,prov_index
0,IX25,April,7987
1,IX25,December,8257
2,IX25,February,8778
3,IX25,January,9291
4,IX25,March,8352


In [20]:
# 数据合并，将总和列合并到省份DF
prov_index2 = pd.merge(prov_index,prov_index_sum,on=("keyword","date"))


In [21]:
# 根据省月份平均搜索指数 / 全国月份平均搜索指数得出搜索所占比
prov_index2['pct'] = prov_index2['prov_index_x'] / prov_index2['prov_index_y']
prov_index2.head()

,id,keyword,prov,prov_index_x,date,prov_index_y,pct
0,1,缤智,913,1000,December,4116,0.242954
1,2,缤智,901,272,December,4116,0.066084
2,3,缤智,917,246,December,4116,0.059767
3,4,缤智,916,234,December,4116,0.056851
4,5,缤智,920,203,December,4116,0.049320


In [22]:
new_index_mean = new_index_mean.reset_index()
new_index_mean

,keyword,date,_index
0,IX25,April,29144.0
1,IX25,December,32422.0
2,IX25,February,28511.0
3,IX25,January,32204.0
4,IX25,June,882.0
5,IX25,March,30081.0
6,IX25,May,27164.0
7,IX25,November,30810.0
8,IX25,October,31611.0
9,T-CROSS,April,150414.0


In [24]:
# 将真实全国搜索指数合并到省份表
prov_index_final = pd.merge(prov_index2,new_index_mean,on=('keyword','date'))


In [25]:
# 省份占比 * 真实搜索指数 = 省份真实搜索指数
prov_index_final['real_prov_index'] = prov_index_final['pct'] * prov_index_final['_index']
prov_index_final.head()

,id,keyword,prov,prov_index_x,date,prov_index_y,pct,_index,real_prov_index
0,1,缤智,913,1000,December,4116,0.242954,82882.0,20136.540330
1,2,缤智,901,272,December,4116,0.066084,82882.0,5477.138970
2,3,缤智,917,246,December,4116,0.059767,82882.0,4953.588921
3,4,缤智,916,234,December,4116,0.056851,82882.0,4711.950437
4,5,缤智,920,203,December,4116,0.049320,82882.0,4087.717687


In [26]:
# 省份在这里是以ID的形式展示出来，需要将其与省份ID对照表合并
prov_index_final = pd.merge(prov_index_final,prov_id,left_on="prov",right_on="id")


In [27]:
# 输出成excel
prov_index_final.to_excel('prov_index_final.xlsx')

